In [1]:
#Data loading
import pandas as pd
dataset_with_null = pd.read_excel("C:\\Users\\91628\\Desktop\\smlprojectcleaneddataset (1).xlsx")
dataset_with_null['days_to_trend'] = dataset_with_null['trending_Date']-dataset_with_null['published_date']
data_og = dataset_with_null.dropna()

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91628\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter
import re
import string

dataset = data_og
def preprocess_text(text):
    # Remove non-ASCII characters
    text = ''.join([char for char in text if char in string.printable])
    return text

def remove_stopwords(text, stop_words):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def process_dataframe(dataset, stop_words):
    # Apply preprocessing to the 'title' column
    dataset['title'] = dataset['title'].apply(preprocess_text)

    # Remove stopwords from the 'title' column
    dataset['title'] = dataset['title'].apply(lambda x: remove_stopwords(x, stop_words))

    return dataset

stop_words = set(stopwords.words('english')) | {'|','1','2','3','4','5','6','7','8','9','0','vs','le','la','de','none'}
dataset = process_dataframe(dataset, stop_words)

In [4]:
#get frequencies of each word
import pandas as pd
from collections import Counter
import re

data_test = dataset[:100]

def replace_with_word_frequencies(df, column_name):
    # Combine all titles into a single string
    all_titles = ' '.join(df[column_name].astype(str).tolist())

    # Tokenize the text into words
    words = re.findall(r'\b\w+\b', all_titles.lower())

    # Calculate word frequencies using Counter for the entire column
    word_frequencies = Counter(words)

    # Function to replace words with their total occurrences in the column
    def replace_with_frequency(title):
        title_words = re.findall(r'\b\w+\b', str(title).lower())
        frequencies = [word_frequencies[word] for word in title_words]
        return frequencies

    # Apply the function to the column and create a new column
    new_column_name = f"{column_name}_frequencies"
    df[new_column_name] = df[column_name].apply(replace_with_frequency)

    return df

dataset = replace_with_word_frequencies(dataset, 'title')

In [5]:
import pandas as pd
from collections import Counter
import re

data_test = dataset[:100]

def replace_with_word_frequencies(df, column_name):
    # Combine all titles into a single string
    all_titles = ' '.join(df[column_name].astype(str).tolist())

    # Tokenize the text into words
    words = re.findall(r'\b\w+\b', all_titles.lower())

    # Calculate word frequencies using Counter
    word_frequencies = Counter(words)

    # Function to replace words with their frequencies
    def replace_with_frequency(title):
        title_words = re.findall(r'\b\w+\b', str(title).lower())
        frequencies = [word_frequencies[word] for word in title_words]
        return frequencies

    # Apply the function to the column and create a new column
    new_column_name = f"{column_name}_frequencies"
    df[new_column_name] = df[column_name].apply(replace_with_frequency)

    # Calculate the sum of word frequencies for each row
    df[f'{column_name}_frequency_sum'] = df[new_column_name].apply(lambda x: sum(x))
    #df[f'{column_name}_frequency_avg'] = df[new_column_name].apply(lambda x: sum(word_frequencies.get(word, 0) for word in re.findall(r'\b\w+\b', ' '.join(map(str, x)).lower())) / len(re.findall(r'\b\w+\b', ' '.join(map(str, x)).lower())) if len(re.findall(r'\b\w+\b', ' '.join(map(str, x)).lower())) > 0 else 0)


    return df

dataset = replace_with_word_frequencies(dataset, 'title')

In [6]:
import pandas as pd

def add_tag_occurrence_list_column(df, column_name):
    # Split the tags in each row and count their occurrences
    all_tags = '|'.join(df[column_name]).split('|')
    tag_counts = pd.Series(all_tags).value_counts()

    # Function to get the total occurrence of a tag in the entire column
    def get_tag_occurrence(tags):
        tag_list = tags.split('|')
        return [tag_counts.get(tag, 0) for tag in tag_list]

    # Create a new column 'tag_occurrence_list' with the total occurrence of tags in each row
    df['tag_frequency_list'] = df[column_name].apply(get_tag_occurrence)

    # Modify 'tag_occurrence_list' to contain the sum of values in each list
    df['tag_frequency_sum'] = df['tag_frequency_list'].apply(lambda x: sum(x))

    return df

# Call the function with your DataFrame and column name
dataset = add_tag_occurrence_list_column(dataset, 'tags')

#preprocess the x and Y values

X = dataset[['categoryId', 'likes', 'dislikes', 'comment_count', 'days_to_trend', 'tag_frequency_sum', 'title_frequency_sum']]
Y = dataset['view_count']

Y

In [7]:
print(type(dataset['days_to_trend'][0]))
dataset['days_to_trend'] = dataset['days_to_trend'].dt.days
type(dataset['days_to_trend'][0])

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


numpy.int64

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score


features = ['likes', 'dislikes', 'comment_count', 'categoryId','title_frequency_sum','tag_frequency_sum','days_to_trend']
target = 'view_count'
X = dataset[features]
y = dataset[target]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42)
X_tune, X_test, y_tune, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
dt_model = DecisionTreeRegressor()

# Fit the model to the training data
dt_model.fit(X_train, y_train)

DecisionTreeRegressor()

In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

param_grid = {

    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]

}

# Use GridSearchCV for hyperparameter tuning with cross-validation

grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_tune, y_tune)
best_dt_model = grid_search.best_estimator_

# Predict on the test set using the best model

y_pred_test = best_dt_model.predict(X_test)

# Evaluate the best model on the test set

r2_test = r2_score(y_test, y_pred_test)

print(f'R^2 Score on Test Set: {r2_test}')

# Print the best parameters

print("Best Parameters:", grid_search.best_params_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
R^2 Score on Test Set: 0.8190771474595037
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5}


In [10]:
#with feature scaling
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
X.columns = X.columns.astype(str)

# Split the Data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42)
X_tune, X_test, y_tune, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_tune_scaled = scaler.transform(X_tune)
X_test_scaled = scaler.transform(X_test)

dt_model = DecisionTreeRegressor()
dt_model.fit(X_train_scaled, y_train)

y_pred_test = dt_model.predict(X_test_scaled)

# Evaluate the model
r2_test = r2_score(y_test, y_pred_test)
print(f'R^2 Score on Test Set: {r2_test}')

R^2 Score on Test Set: 0.8425464806396569
